In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [68]:
# Korean
plt.rcParams['font.family'] = 'NanumGothic'

In [87]:
#load data
data = pd.read_csv('merged_data.csv', encoding='utf-8')

In [90]:
data.head()

,stn_id,borrowed_hour,borrowed_day,is_holiday,borrowed_num_nearby,강수량(mm),wind_chill,stn_gu,nearby_id,borrowed_date,borrowed_num
0,ST-10,0,6,1,2,0.0,21.031237,마포구,ST-2167,20221001,2
1,ST-10,1,6,1,1,0.0,20.222173,마포구,ST-2167,20221001,2
2,ST-10,2,6,1,2,0.0,19.589061,마포구,ST-2167,20221001,1
3,ST-10,3,6,1,0,0.0,19.552158,마포구,ST-2167,20221001,0
4,ST-10,4,6,1,2,0.0,18.783459,마포구,ST-2167,20221001,0


In [70]:
data.size

106121818

In [71]:
sample_size = 10000  # 원하는 표본 크기
sampled_data = data.sample(n=sample_size, random_state=42)

In [72]:
# 데이터 전처리
X_sampled = sampled_data.drop(['stn_id', 'borrowed_date', 'borrowed_num'], axis=1)
y_sampled = sampled_data['borrowed_num']

In [73]:
X_sampled = pd.get_dummies(X_sampled, columns=['stn_gu', 'nearby_id'])

In [62]:
# 훈련 세트와 테스트 세트로 데이터 분할
X_train_sampled, X_test_sampled, y_train_sampled, y_test_sampled = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42)

In [63]:
# XGBoost 모델 생성과 훈련
model_sampled = XGBRegressor()
model_sampled.fit(X_train_sampled, y_train_sampled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

# Overfitting Checking

In [80]:
# 데이터 샘플링
sample_size = 100000
sampled_data = data.sample(n=sample_size, random_state=42)


In [81]:
# 데이터 전처리
X_sampled = sampled_data.drop(['stn_id', 'borrowed_date', 'borrowed_num'], axis=1)
y_sampled = sampled_data['borrowed_num']
X_sampled = pd.get_dummies(X_sampled, columns=['stn_gu', 'nearby_id'])

In [82]:
# 훈련 세트와 검증 세트로 데이터 분할
X_train_sampled, X_val_sampled, y_train_sampled, y_val_sampled = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42)

In [83]:
# XGBoost 모델 생성과 훈련
model_sampled = XGBRegressor()
model_sampled.fit(X_train_sampled, y_train_sampled)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [84]:
# 훈련 데이터와 검증 데이터에 대한 예측
y_train_pred = model_sampled.predict(X_train_sampled)
y_val_pred = model_sampled.predict(X_val_sampled)

In [85]:
# 훈련 데이터와 검증 데이터에 대한 MSE, MAE, RMSE 계산
mse_train = mean_squared_error(y_train_sampled, y_train_pred)
mae_train = mean_absolute_error(y_train_sampled, y_train_pred)
rmse_train = np.sqrt(mse_train)

mse_val = mean_squared_error(y_val_sampled, y_val_pred)
mae_val = mean_absolute_error(y_val_sampled, y_val_pred)
rmse_val = np.sqrt(mse_val)

print('Training Metrics:')
print(f'MSE: {mse_train}, MAE: {mae_train}, RMSE: {rmse_train}')

print('\nValidation Metrics:')
print(f'MSE: {mse_val}, MAE: {mae_val}, RMSE: {rmse_val}')

Training Metrics:
MSE: 3.6922576853073448, MAE: 1.183427960601468, RMSE: 1.9215248333829424

Validation Metrics:
MSE: 5.8181392164156565, MAE: 1.2831100866649883, RMSE: 2.4120819257263335


## Sampled Data Model Evaluation

In [64]:
# 모델 평가
y_pred_sampled = model_sampled.predict(X_test_sampled)
mse_sampled = mean_squared_error(y_test_sampled, y_pred_sampled)
print(f'Mean Squared Error on Sampled Data: {mse_sampled}')

Mean Squared Error on Sampled Data: 4.608116875914985


In [65]:
mae_sampled = mean_absolute_error(y_test_sampled, y_pred_sampled)
print(f'Mean Absolute Error on Sampled Data: {mae_sampled}')

Mean Absolute Error on Sampled Data: 1.2380934204390803


In [66]:
rmse_sampled = np.sqrt(mean_squared_error(y_test_sampled, y_pred_sampled))
print(f'Root Mean Squared Error on Sampled Data: {rmse_sampled}')

Root Mean Squared Error on Sampled Data: 2.146652481403309


# Total Data

In [16]:
# 데이터 전처리
X = data.drop(['stn_id', 'borrowed_date', 'borrowed_num'], axis=1)  # 불필요한 열 제거
y = data['borrowed_num']

In [17]:
# 범주형 변수를 더미 변수로 변환 (예: stn_gu)
X = pd.get_dummies(X, columns=['stn_gu', 'nearby_id'])

In [18]:
# 훈련 세트와 테스트 세트로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# XGBoost 모델 생성과 훈련
model = XGBRegressor()
model.fit(X_train, y_train)

MemoryError: Unable to allocate 121. GiB for an array with shape (2108, 7717950) and data type float64

In [23]:
model = XGBRegressor(
    max_depth=6,          # 트리의 최대 깊이
    subsample=0.8,        # 각 트리에 사용되는 데이터의 일부분
    colsample_bytree=0.8  # 각 트리에 사용되는 특징의 일부분
)

model.fit(X_train, y_train)

MemoryError: Unable to allocate 121. GiB for an array with shape (2108, 7717950) and data type float64

In [ ]:
# 모델 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')